# Word embeddings
## WE en spaCy
Los word embeddings (o word vectors) son representaciones numéricas de las palabras, generadas con una reducción de dimensionalidad sobre una matriz de co-ocurrencia sobre un corpus enorme. Spacy utiliza los word vectors de GloVe, (*Stanford's Global Vectors for Word Representation*). Estos vectores se pueden utilizar para calcular la similaridad semántica entre palabras o documentos.

El vocabulario por defecto en el modelo spaCy del idioma inglés (`en_core_web_sm`) es muy pequeño. Hay que cargar en_core_web_md (`python -m spacy download en_core_web_md`) para tener un conjunto de word vectors mayor. El modelo de tamaño medio en español (`python -m spacy download es_core_news_md`) contiene vectores también.

In [ ]:
import spacy
import numpy as np

nlp = spacy.load("es_core_news_md")

In [ ]:
nlp.vocab.vectors

In [ ]:
len(nlp.vocab.vectors)

In [ ]:
nlp.vocab.vectors_length

In [ ]:
madrid = nlp.vocab["Madrid"]
madrid.vector.shape

In [ ]:
type(madrid)

In [ ]:
doc = nlp("me voy a Madrid")

In [ ]:
doc[3]

In [ ]:
type(doc[3])

El vector del lexema 'Madrid' es igual al token 'Madrid'

In [ ]:
sum(madrid.vector == doc[3].vector)

In [ ]:
madrid.vector[:10]

In [ ]:
nlp.vocab.get_vector("Madrid")[:10]

In [ ]:
doc[3].vector[:10]

Similitudes

In [ ]:
toledo = nlp.vocab["Toledo"]
madrid.similarity(toledo)

In [ ]:
manzana = nlp.vocab["manzana"]
madrid.similarity(manzana)

In [ ]:
manzana.similarity(nlp.vocab["pera"])

### Visualización de word embeddings

In [ ]:
len(nlp.vocab)

In [ ]:
lexemas = [nlp.vocab[orth] for orth in nlp.vocab.vectors]

In [ ]:
len(lexemas)

In [ ]:
lexemas_vector = [nlp.vocab[orth] for orth in nlp.vocab.vectors if nlp.vocab[orth].has_vector]

In [ ]:
len(lexemas_vector)

In [ ]:
words = [t.text for t in np.random.choice(lexemas, 25, replace=False)]
word_vectors = np.array([nlp(word).vector for word in words])

words

### Visualización t-SNE

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(n_components=2, random_state=0, n_iter=10000, perplexity=2, init='random', learning_rate='auto')
np.set_printoptions(suppress=True)
T = tsne.fit_transform(word_vectors)
labels = words
plt.figure(figsize=(14, 8))
plt.scatter(T[:, 0], T[:, 1], c='steelblue', edgecolors='k')
for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

### Visualización PCA

In [ ]:
word_vectors = [t.vector for t in np.random.choice(lexemas, 10000, replace=False)]

In [ ]:
palabras = ['manzana', 'pera', 'Madrid', 'Toledo']

In [ ]:
palabras_vectors = np.array([nlp(word).vector for word in palabras])

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
np.set_printoptions(suppress=True)
T = pca.fit_transform(word_vectors)

plt.figure(figsize=(14, 8))
plt.scatter(T[:, 0], T[:, 1], c='steelblue',alpha=0.05)

labels = palabras
T = pca.transform(palabras_vectors)
plt.scatter(T[:, 0], T[:, 1], c='lime', edgecolors='darkgreen')

for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

### Visualización t-SNE extendida

In [ ]:
palabras_all = [t.text for t in np.random.choice(lexemas, 10000, replace=False)] + palabras

In [ ]:
palabras_vectors = np.array([nlp(word).vector for word in palabras_all])

In [ ]:
tsne = TSNE(n_components=2, random_state=0, n_iter=250, perplexity=5, init='random', learning_rate='auto')
np.set_printoptions(suppress=True)
T = tsne.fit_transform(palabras_vectors)

plt.figure(figsize=(14, 8))
plt.scatter(T[:, 0], T[:, 1], c='steelblue', alpha=0.05)

labels = palabras

plt.scatter(T[-len(palabras):, 0], T[-len(palabras):, 1], c='lime', edgecolors='darkgreen')
for label, x, y in zip(labels, T[-len(palabras):, 0], T[-len(palabras):, 1]):
    plt.annotate(label, xy=(x+0.1, y+0.1), xytext=(0, 0), textcoords='offset points')

# Word embeddings con Gensim
Cargamos un conjunto de WE ya pre-entrenado con la API de Gensim:\
https://radimrehurek.com/gensim/downloader.html

In [ ]:
import gensim.downloader as api
list(api.info()['models'].keys())

In [ ]:
api.info('glove-twitter-50')

In [ ]:
for model_name, model_data in sorted(api.info()['models'].items()):
    print(f"""{model_name} ({model_data.get('num_records', "None")} records):
    {model_data['description']}\n""")

In [ ]:
#cargamos el modelo deseado con
model = api.load("glove-wiki-gigaword-50")
model

Podemos usar los modelos cargados para ver los vectores de una palabra, buscar palabras similares o calcular analogías.\
Los modelos cargados son objetos de clase `models.keyedvectors` (https://radimrehurek.com/gensim/models/keyedvectors.html)

In [ ]:
dir(model)

Podemos listar todas las palabras del modelo

In [ ]:
palabras = model.index_to_key

np.random.choice(palabras, 10)

Cada palabra tiene su vector

In [ ]:
vec_king = model['king']

In [ ]:
type(vec_king)

In [ ]:
vec_king.shape

In [ ]:
len(model.vectors)

In [ ]:
palabra_rara = 'zamburiña'
try:
    vector = model[palabra_rara]
except KeyError:
    print(f"La palabra '{palabra_rara}' no aparece en este modelo")

In [ ]:
model.most_similar("apple")

In [ ]:
model.most_similar("bank")

In [ ]:
model.similarity('apple','pear')

In [ ]:
model.similarity('banana','pear')

In [ ]:
print(model.doesnt_match("apple pear banana city".split())) #palabra que no encaja en el contexto del resto

### Analogías de word vectors con Gensim
![rel](word2vec-king-queen-composition.png)  
Si *hombre* es a *rey*, entonces *mujer* es a *??*\
Se calcula como la palabra más cercana al vector (rey - hombre) + mujer   

In [ ]:
# hombre es a rey como mujer es a XX
# rey - hombre + mujer 
#https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar_cosmul
model.most_similar(positive=['king','woman'],negative=['man'])

In [ ]:
word_vectors = [model[t] for t in np.random.choice(model.index_to_key, 10000, replace=False)]

In [ ]:
palabras = ['man', 'woman', 'king', 'queen', 'son', 'daughter']

In [ ]:
palabras_vectors = np.array([model[word] for word in palabras])

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
np.set_printoptions(suppress=True)
T = pca.fit_transform(word_vectors)

plt.figure(figsize=(14, 8))
plt.scatter(T[:, 0], T[:, 1], c='steelblue',alpha=0.05)

labels = palabras
T = pca.transform(palabras_vectors)
plt.scatter(T[:, 0], T[:, 1], c='lime', edgecolors='darkgreen')

for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')

### Carga de otros modelos pre-entrenados en Gensim
En lugar de usar su API cargamos los modelos en formato texto. Hay varios modelos en Español en https://github.com/dccuchile/spanish-word-embeddings

In [ ]:
#carga de vectores en formato TXT
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec = '~/Downloads/fasttext-sbwc.100k.vec'
cantidad = 100000
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [ ]:
wordvectors

In [ ]:
wordvectors['rey'].shape

In [ ]:
len(wordvectors.vectors)

In [ ]:
wordvectors.most_similar(positive=['rey','mujer'],negative=['hombre'], topn=3)

In [ ]:
wordvectors.most_similar(positive=['yerno','mujer'],negative=['hombre'], topn=3)

In [ ]:
# correr -> corrían como saltar -> XX
wordvectors.most_similar(positive=['corrían','saltar'],negative=['correr'], topn=3)

In [ ]:
# Francia -> París como España -> XX
wordvectors.most_similar(positive=['parís','españa'],negative=['francia'], topn=3)

### Modelos FastText
Los modelos de FastText se pueden cargar en formato texto (sólo palabras pre-entrenadas) o como modelo binario (calcula nuevas palabras a partir de su n-grama de caracteres)  
Modelos pre-entrenados de FastText: https://github.com/mquezada/starsconf2018-word-embeddings

In [ ]:
wordvectors.most_similar(['adiós'])

In [ ]:
palabra_rara = 'pequeñín'
try:
    vector = wordvectors[palabra_rara]
except KeyError:
    print(f"La palabra '{palabra_rara}' no aparece en este modelo")

In [ ]:
del(wordvectors)

In [ ]:
# vectores de FastText desde el formato binario (lento, requiere mucha memoria)
# descargado de https://fasttext.cc/docs/en/crawl-vectors.html
# ¡ojo, ocupan 4,5 GB!
from gensim.models.fasttext import load_facebook_vectors

wordvectors_file = '/Users/jovifran/Downloads/cc.es.300.bin'
wordvectors = load_facebook_vectors(wordvectors_file) #carga vectores pre-entrenados sólo

In [ ]:
wordvectors

In [ ]:
wordvectors['adiós'].shape

In [ ]:
len(wordvectors.vectors)

In [ ]:
'pequeñín' in wordvectors.key_to_index

In [ ]:
wordvectors.most_similar('pequeñín')

In [ ]:
#'neorevolucionario' in wordvectors.vocab #versión gensim <4.0
'neorevolucionario' in wordvectors.key_to_index #veersión gensim >=4.0

In [ ]:
wordvectors['neorevolucionario'][:10]

In [ ]:
wordvectors.most_similar('neorevolucionario')

Se puede usar como corrector ortográfico

In [ ]:
'ayedo' in wordvectors.key_to_index

In [ ]:
wordvectors.most_similar('ayedo')

In [ ]:
del wordvectors

## Entrenamiento de vectores propios
En lugar de usar vectores preentrenados los podemos entrenar con el modelo `word2vec` de Gensim

In [ ]:
import spacy
nlp = spacy.load('es_core_news_md')

In [ ]:
def normalizar_doc_tokenize(doc):
    '''Función que normaliza un texto cogiendo sólo
    las palabras en minúsculas mayores de 3 caracteres'''
    # separamos en tokens
    tokens = nlp.make_doc(doc)
    # filtramos stopwords
    filtered_tokens = [t.lower_ for t in tokens if
                       len(t.text)>3 and
                       not t.is_space and
                       not t.is_punct]

    return filtered_tokens

In [ ]:
with open('cañas y barro.txt', 'r', encoding = 'utf-8') as f:
    texto = f.readlines()
TOKENIZED_CORPUS = list(map(normalizar_doc_tokenize, texto))
len(TOKENIZED_CORPUS)

Consideramos cada línea como un documento completo

In [ ]:
TOKENIZED_CORPUS[100]

In [ ]:
texto[100]

Calculamos los vectores de las palabras de nuestro corpus

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(TOKENIZED_CORPUS, #lista de documentos como lista de tokens
                               vector_size=50,          #tamaño del vector
                               window=5,         #nº de términos adyacentes que usamos para el cálculo
                               min_count=10,      #nº mínimo de apariciones del término para contarlo
                               epochs = 50)

#una vez entrenado el modelo nos quedamos con los vectores calculados
#si no se van a actualizar los vectores con nuevos documentos
model = model.wv
len(model.index_to_key)

In [ ]:
type(model)

Este modelo funciona como los que hemos utilizado anteriormente en la librería `gensim`

# Vectores de documento (modelos semánticos)
Los vectores de documento recogen el sentido semántico de todo el documento como un vector de dimensines únicas.
## Modelos basados en *word embeddings*
Calcula el promedio de los *word embeddings* del documento para obtener un vector con sentido semántico de todo el documento.

In [ ]:
import spacy
nlp = spacy.load('es_core_news_md')

#Librería spaCy
#El atributo vector del Doc o Span calcula el promedio de sus vectores de palabra

doc1 = nlp("Me gustan las patatas fritas y las hamburguesas.")
doc2 = nlp("La comida rápida sabe muy bien.")

In [ ]:
type(doc1[0])

In [ ]:
doc1[0].vector.shape

In [ ]:
type(doc1)

En la librería `spaCy` el objeto `Doc` contiene el vector del documento como promedio de sus `word embeddings`

In [ ]:
doc1.vector.shape

In [ ]:
type(doc1[2:4])

In [ ]:
doc1[2:4].vector.shape

In [ ]:
# Similitud de dos documentos
print(doc1, "<->", doc2, doc1.similarity(doc2))
# Similitud de tokens y spans
patatas_fritas = doc1[3:5]
hamburguesas = doc1[7]
print(patatas_fritas, "<->", hamburguesas, patatas_fritas.similarity(hamburguesas))

In [ ]:
comida_rapida = doc2[1:3]
print(patatas_fritas, "<->", comida_rapida, patatas_fritas.similarity(comida_rapida))

In [ ]:
muy_bien = doc2[4:6]
print(patatas_fritas, "<->", muy_bien, patatas_fritas.similarity(muy_bien))

### Modelo Sentence embeddings (SBERT)
Usamos un modelo *transformer* entrenado en texto en inglés.

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

sentences = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome',
             'The dog plays in the garden',
             'A woman listens to music',
             'The new movie is so great']

#Compute embedding
embeddings = model.encode(sentences, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings, embeddings)

In [ ]:
embeddings.shape

In [ ]:
cosine_scores

In [ ]:
for i, s1 in enumerate(sentences):
    for j, s2 in enumerate(sentences[i+1:]):
        print(f"Score: {cosine_scores[i][i+j+1]:.4f}:\t{s1}, {s2}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(embeddings[0].reshape(-1,384).cpu(),cmap="Greys",center=0,square=False)
plt.gcf().set_size_inches(10,1)
plt.axis('off')
plt.title(sentences[0])
plt.show()

sns.heatmap(embeddings[2].reshape(-1,384).cpu(),cmap="Greys",center=0,square=False)
plt.gcf().set_size_inches(10,1)
plt.axis('off')
plt.title(sentences[2])
plt.show()

sns.heatmap(embeddings[5].reshape(-1,384).cpu(),cmap="Greys",center=0,square=False)
plt.gcf().set_size_inches(10,1)
plt.axis('off')
plt.title(sentences[5])
plt.show()

In [ ]:
#modelo multilingüe
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')


In [ ]:
sentences = ['el gato juega en el jardín',
             'A man is playing guitar',
             'The new movie is awesome',
             'The dog plays in the garden',
             'una mujer escucha música',
             'la nueva película es asombrosa']

#Compute embedding
embeddings = model.encode(sentences, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings, embeddings)

In [ ]:
for i, s1 in enumerate(sentences):
    for j, s2 in enumerate(sentences[i+1:]):
        print(f"Score: {cosine_scores[i][i+j+1]:.4f}:\t{s1}, {s2}")

In [ ]:
sns.heatmap(embeddings[2].reshape(-1,384).cpu(),cmap="Greys",center=0,square=False)
plt.gcf().set_size_inches(10,1)
plt.axis('off')
plt.title(sentences[2])
plt.show()

sns.heatmap(embeddings[5].reshape(-1,384).cpu(),cmap="Greys",center=0,square=False)
plt.gcf().set_size_inches(10,1)
plt.axis('off')
plt.title(sentences[5])
plt.show()